# MC56F81xxx/MWC20D2X Device HSM and Secure Boot

This notebook describes how to setup basic secure boot on MC56F81xxx/MWC20D2X devices using the SPSDK *nxpdevhsm* and *nxpimage* applications.

Prepare a plain binary image as an input for the *nxpimag* tool. In our example we use simple led blinky image.


In [1]:
%run ../init_notebook.ipynb

import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

WORKSPACE = "workspace/" # change this to path to your workspace
VERBOSITY = "-v" # verbosity of commands, might be -v or -vv for debug or blank for no additional info

BINARY_FILE = WORKSPACE + "mc56_flash_image.bin"


env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


# Signed MBI

Let's generate signed MBI first from the template.

![Signed Image Layout](img/image_layout.png "Signed Image Layout")

### Certificate block
First step is to create certificate block and then MBI

Certificate block configuration contains ISK certificate configuration and signing key

```yaml
# =====================================  Certification Block for MC56xx template  ======================================

# ======================================================================================================================
#                                             == ISK Certificate Settings ==                                            
# ======================================================================================================================
# ------------------------------===== Use ISK for signature certification [Required] =====------------------------------
# Description: Enable ISK type of signature certification. Unused when 'binaryCertificateBlock' is defined
useIsk: false
# ---------------------------------===== ISK public key [Conditionally required] =====----------------------------------
# Description: Path to ISK public key.
iskPublicKey: sign_cert.pem
# -----------------------===== Main root Certification Private Key [Conditionally required] =====-----------------------
# Description: Path to Main root Certification Private Key.
signPrivateKey: main_cert_prv_key.pem
# -------------------------------===== Signature Provider [Conditionally required] =====--------------------------------
# Description: Signature provider configuration in format 'type=<sp_type>;<key1>=<value1>;<key2>=<value2>".
signProvider: type=file;file_path=root_prv_key.pem
# ======================================================================================================================
#                                                  == Basic Settings ==                                                 
# ======================================================================================================================
# --------------------------------------===== cert block filename [Required] =====--------------------------------------
# Description: Generated cert block filename.
containerOutputFile: cert_block.bin


```

In [4]:
# generate template for cert block
CERT_BLOCK_TEMPLATE = WORKSPACE + "cert_block_template.yaml"
# generate template for mbi
TEMPLATES_PATH = WORKSPACE + "templates"
# choose family for the MCU
FAMILY = "mc56f81xxx"

# generate template for cert block
%! nxpimage $VERBOSITY cert-block get-template -f $FAMILY -o $CERT_BLOCK_TEMPLATE
# generate template for MBI
%! nxpimage $VERBOSITY mbi get-templates -f $FAMILY -o $TEMPLATES_PATH --force
# just for verification that the template was generated
assert os.path.exists(os.path.join(TEMPLATES_PATH, "mc56f81xxx_xip_signed.yaml"))
assert os.path.exists(CERT_BLOCK_TEMPLATE)

nxpimage -v cert-block get-template -f mc56f81xxx -o workspace/cert_block.yaml
Creating /Users/macbook-m1/spsdk_test/spsdk/examples/jupyter_examples/mc56/workspace/cert_block.yaml template file.
nxpimage -v mbi get-templates -f mc56f81xxx -o workspace/templates --force
Creating /Users/macbook-m1/spsdk_test/spsdk/examples/jupyter_examples/mc56/workspace/templates/mc56f81xxx_xip_signed.yaml template file.


Our configuration might look like this

```yaml
# =================  Master Boot Image Configuration template for mc56f81xxx, Plain Signed XIP Image.  =================

# ======================================================================================================================
#                                                  == Basic Settings ==                                                 
# ======================================================================================================================
# ------------------------------------------===== MCU family [Required] =====-------------------------------------------
# Description: MCU family name.
# Possible options: <mc56f81xxx, mwct20d2x>
family: mc56f81xxx
# --------------------------------------===== Application target [Required] =====---------------------------------------
# Description: Definition if application is Execute in Place(XiP) or loaded to RAM during reset sequence.
# Possible options: <xip, load-to-ram>
outputImageExecutionTarget: xip
# -------------------------------===== Type of boot image authentication [Required] =====-------------------------------
# Description: Specification of final master boot image authentication.
# Possible options: <plain, crc, signed, signed-encrypted, signed-nxp>
outputImageAuthenticationType: signed
# ------------------------------------===== Master Boot Image name [Required] =====-------------------------------------
# Description: The file for Master Boot Image result file.
masterBootOutputFile: my_mbi.bin
# ------------------------------------===== Plain application image [Required] =====------------------------------------
# Description: The input application image to by modified to Master Boot Image.
inputImageFile: my_application.bin
# ---------------------------------------===== Firmware version. [Optional] =====---------------------------------------
# Description: Value compared with Secure_FW_Version monotonic counter value stored in PFR/IFR. If value is lower than
# value in PFR/IFR, then is image rejected (rollback protection)..
firmwareVersion: 0
# ======================================================================================================================
#                                               == Certificate Block vX ==                                              
# ======================================================================================================================
# -----------------------------===== Certificate Block binary/config file [Required] =====------------------------------
# Description: Path to certificate block binary or config file.
certBlock: cert_block.bin
# ======================================================================================================================
#                                              == Image Signing Settings ==                                             
# ======================================================================================================================
# --------------------------===== Main Certificate private key [Conditionally required] =====---------------------------
# Description: Main Certificate private key used to sign certificate. It can be replaced by signProvider key.
signPrivateKey: main_prv_key.pem
# -------------------------------===== Signature Provider [Conditionally required] =====--------------------------------
# Description: Signature provider configuration in format 'type=<sp_type>;<key1>=<value1>;<key2>=<value2>".
# signProvider: type=file;file_path=my_prv_key.pem

```


In [8]:
MBI_CONFIG_PATH = WORKSPACE + "mbi_xip.yaml"
CERT_BLOCK_CONFIG_PATH = WORKSPACE + "cert_block.yaml"


# export cert block and Master Boot Image
%! nxpimage $VERBOSITY cert-block export -c $CERT_BLOCK_CONFIG_PATH -f $FAMILY
%! nxpimage $VERBOSITY mbi export -c $MBI_CONFIG_PATH


# Split MBI into two images at offset 0x0800
%! nxpimage  utils binary-image extract -b ./workspace/mb_xip_signed.bin -a 0 -s 0x800 -o ./workspace/mb_xip_signed_0_0x800.bin
%! nxpimage  utils binary-image extract -b ./workspace/mb_xip_signed.bin -a 0x800 -s 0 -o ./workspace/mb_xip_signed_0x800.bin


nxpimage -v cert-block export -c workspace/mbi_xip.yaml -f mc56f81xxx


ERROR:SPSDK: Configuration validation failed: data must be valid exactly by one definition (0 matches found)
Reason of fail for oneOf rule#0: 
 data must contain ['signingCertificateFile'] properties; Missing field(s): signingCertificateFile

Reason of fail for oneOf rule#1: 
 data must contain ['iskPublicKey'] properties; Missing field(s): iskPublicKey

You need to define oneOf of the following sets: ['signingCertificateFile'] ['iskPublicKey']
nxpimage -v mbi export -c workspace/mbi_xip.yaml
ERROR:SPSDK: Unable to load 'cert_block.bin'.
nxpimage utils binary-image extract -b ./workspace/mb_xip_signed.bin -a 0 -s 0x800 -o ./workspace/mb_xip_signed_0_0x800.bin
Success. (Extracted chunk: /Users/macbook-m1/spsdk_test/spsdk/examples/jupyter_examples/mc56/workspace/mb_xip_signed_0_0x800.bin created.)
nxpimage utils binary-image extract -b ./workspace/mb_xip_signed.bin -a 0x800 -s 0 -o ./workspace/mb_xip_signed_0x800.bin
Success. (Extracted chunk: /Users/macbook-m1/spsdk_test/spsdk/examples/

# Device HSM OEM Provisioning


![SBX Structure ](img/sbx.png "SBX Structure ")
Create OEM provisioning SBx file using the nxpdevhsm application

Our configuration might look like this

```yaml
# ===========  Secure Binary X Configuration template for mc56f81xxx.  ===========
# ----------------------------------------------------------------------------------------------------
#                                         == Basic Settings ==
# ----------------------------------------------------------------------------------------------------
firmwareVersion: 0 # [Optional], Firmware version., Version of application image firmware.
containerOutputFile: workspace/my_new.sbx # [Required], SBx filename, Generated SBx container filename.
family: mc56f81xxx # [Required], MCU family, MCU family name., Possible options:['mc56f81xxx']
# ----------------------------------------------------------------------------------------------------
#                                    == Secure Binary X Settings ==
# ----------------------------------------------------------------------------------------------------
image_type: OEM_PROVISIONING # [Optional], image type, SecureBinaryXType, Possible options:['NXP_PROVISIONING', 'OEM_PROVISIONING', 'OEM']
description: This is description of generated SB file. # [Optional], Description, Description up to 16 characters, longer will be truncated. Stored in SBx manifest.
# ----------------------------------------------------------------------------------------------------
#                               == Secure Binary X Commands Settings ==
# ----------------------------------------------------------------------------------------------------
commands: # [Required], SBx.1 Commands, Secure Binary X commands block, list of all possible options - Modify it according to your application
  # ----------------------------------------------------------------------------------------------------
  #          == List of possible 5 options. Option types[object,object,object,object,object] ==
  # ----------------------------------------------------------------------------------------------------
  - # [Example of possible configuration #0]
    erase: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0 # [Required], Address, Jump-to address to start execute code.
      size: 0x20000

  - # [Example of possible configuration #0]
    load: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0 # [Required], Address, Jump-to address to start execute code.
      file: mb_xip_signed_0_0x800.bin

  - # [Example of possible configuration #0]
    load: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0x0c00 # [Required], Address, Jump-to address to start execute code.
      file: mb_xip_signed_0x800.bin

  # - # [Example of possible configuration #4]
  #   reset: true  # [Required], Reset

```


In [1]:
SBX_WORKSPACE = WORKSPACE + "sbx_ws/"   
COM_PORT = "com15"
SBX_CONFIG = WORKSPACE + "sbx.yaml"
SBX_FILE = WORKSPACE + "my_new.sbx"
OEM_SHARE_INPUT = WORKSPACE + "OEM_SHARE_INPUT.BIN"

%! nxpdevhsm $VERBOSITY generate -f mc56f81xxx -c $SBX_CONFIG -p $COM_PORT -w $SBX_WORKSPACE -i $OEM_SHARE_INPUT

nxpdevhsm -v generate -f mc56f81xxx -c workspace/sbx.yaml -p com15 -w workspace/sbx_ws/ -i workspace/OEM_SHARE_INPUT.BIN 
INFO:spsdk.mboot.mcuboot:Connect: <spsdk.mboot.interfaces.uart.MbootUARTInterface object at 0x000001A63BC26290>
 1: Initial target reset is disabled 
 2: Generating OEM master share.
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x04003000, length=16, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty('MaxPacketSize', index=0)
INFO:spsdk.mboot.mcuboot:CMD: Max Packet Size = 32
INFO:spsdk.mboot.mcuboot:CMD: Successfully Send 16 out of 16 Bytes
INFO:spsdk.mboot.mcuboot:CMD: [TrustProvisioning] DSC HSM CREATE SESSION
INFO:spsdk.mboot.mcuboot:CMD: ReadMemory(address=0x04003100, length=52, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: Successfully Received 52 from 52 Bytes
 5: Encrypting sbx data on device
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x04003000, length=172, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty('MaxPacketSize', index=0)
INFO:spsdk.mboot

In [1]:
# Receive SBx file

%! blhost -p com15 receive-sb-file workspace/my_new.sbx

blhost -p com15 receive-sb-file workspace/my_new.sbx 
Sending SB file
Response status = 0 (0x0) Success.
